In [1]:
import sys
sys.path.append('/workspace/data/textmining_dataset/')
sys.path.append('../')

import gensim
import sklearn
import soynlp
print('gensim  version = {}'.format(gensim.__version__))
print('sklearn version = {}'.format(sklearn.__version__))
print('soynlp  version = {}'.format(soynlp.__version__))

gensim  version = 3.6.0
sklearn version = 0.20.3
soynlp  version = 0.0.492


데이터를 메모리에 올렸습니다.

In [2]:
from lovit_textmining_dataset.navermovie_comments import load_movie_comments
from pprint import pprint

idxs, texts, rates = load_movie_comments(large=True, tokenize='komoran')
pprint(texts[:3])
pprint(rates[:3])

('명불허전/NNP',
 '왠지/MAG 고사/NNP 피/NNG 의/JKG 중간고사/NNG 보다/JKB 재미/NNG 가/JKS 없/VA 을/ETM 듯/NNB '
 '하/XSA 어요/EC 만약/MAG 보/VV 게/EC 되/VV ㄴ다면/EC 실망/NNG 하/XSV ㄹ/ETM 듯/NNB',
 '티아라/NNP 사랑해/NNP ㅜ/NA')
[1, 1, 10]


Doc2Vec 의 document id 는 영화평의 평점입니다.

In [3]:
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

class Dataset:
    def __init__(self, texts, rates):
        self.texts = texts
        self.rates = rates
    def __iter__(self):
        for text, rate in zip(self.texts, self.rates):
            if not text:
                continue
            words = text.split()
            words = [w.split('/', 1)[0] if '/NN' in w else w for w in words]
            yield TaggedDocument(words=words, tags=['#%s' % rate])

# doc2vec = Doc2Vec(Dataset(texts, rates), min_count=10)

In [4]:
import pickle

with open('doc2vec_movie.pkl', 'rb') as f:
    doc2vec = pickle.load(f)

document vectors 의 index 입니다.

In [5]:
idx_to_doc = doc2vec.docvecs.index2entity
idx_to_doc

['#1', '#10', '#8', '#3', '#9', '#6', '#7', '#2', '#4', '#5']

이후에 각 점수 별 term frequency vector 를 만들 것이기 때문에 {점수:index} 를 만듭니다.

In [6]:
rate_to_idx = {int(key[1:]):idx for idx, key in enumerate(idx_to_doc)}
rate_to_idx

{1: 0, 10: 1, 8: 2, 3: 3, 9: 4, 6: 5, 7: 6, 2: 7, 4: 8, 5: 9}

단어 역시 {단어:index} 를 만들 것입니다.

In [7]:
idx_to_vocab = doc2vec.wv.index2word
vocab_to_idx = {vocab:idx for idx, vocab in enumerate(idx_to_vocab)}

print('num vocabs = %d' % len(idx_to_vocab))

num vocabs = 31537


이번에는 31,537 개의 단어 모두가 아닌, 몇 몇 점수에서 유독 많이 등장하는 단어만 선택하여 임베딩을 할 것입니다. 이를 위하여 (점수, 단어) 빈도 행렬을 만듭니다.

In [8]:
from collections import defaultdict
from scipy.sparse import csr_matrix

def make_bow(texts, rates, vocab_to_idx, rate_to_idx, topk=5000):
    # count (score, term)
    score_to_terms = defaultdict(lambda: defaultdict(int))
    for text, rate in zip(texts, rates):
        words = [vocab_to_idx.get(w, -1) for w in text.split()]
        words = [w for w in words if w >= 0 and w < topk]
        if not words:
            continue
        d = rate_to_idx[rate]
        for w in words:
            score_to_terms[d][w] += 1

    # make sparse matrix
    rows, cols, data = [], [], []
    for score, terms in score_to_terms.items():
        for term, count in terms.items():
            rows.append(score)
            cols.append(term)
            data.append(count)
    return csr_matrix((data, (rows, cols)))

bow = make_bow(texts, rates, vocab_to_idx, rate_to_idx)

각 점수별 단어의 등장 비율 (prop) 를 만든 뒤, 문서 전체의 단어 등장 비율(prop_total) 을 만듭니다. 그리고 prop / (prop + prop_total) 을 이용하여 점수 별 키워드 점수를 계산합니다. 이는 마치 Point Mutual Information 과도 비슷합니다만, infrequent words 에 덜 민감하게 작동하며, [0, 1] 로 bounded 된 키워드 점수를 만들어줍니다. 점수가 0.5 라면 전체 데이터에서의 등장 비율이나 한 점수에서의 등장비율이 같다는 의미입니다. 1 에 가까워질수록 특정 점수에서 자주 등장했다는 의미입니다.

In [9]:
import numpy as np
from sklearn.preprocessing import normalize

prop = normalize(bow, norm='l1')
prop_total = normalize(bow.sum(axis=0), norm='l1')
ratio = prop / (prop + prop_total)
ratio = np.nan_to_num(ratio)

print(bow.shape)
print(prop.shape)
print(prop_total.shape)
print(ratio.shape)

(10, 4997)
(10, 4997)
(1, 4997)
(10, 4997)


/home/lovit/anaconda3/envs/jupyter/lib/python3.6/site-packages/scipy/sparse/base.py:594: RuntimeWarning: divide by zero encountered in true_divide
  return np.true_divide(self.todense(), other)
/home/lovit/anaconda3/envs/jupyter/lib/python3.6/site-packages/scipy/sparse/base.py:594: RuntimeWarning: invalid value encountered in true_divide
  return np.true_divide(self.todense(), other)


이 점수가 0.6 이상이며, 0.85 이하인 단어들을 선택했습니다. 1 에 매우 가까우면 infrequent 할 가능성이 높기 때문입니다. 이렇게 선택된 단어는 1525 개 입니다.

In [10]:
rows, cols = np.where( (ratio > 0.6) & (ratio < 0.85) )
keywords = np.unique(cols)
sample_to_vocab = [idx_to_vocab[idx] for idx in keywords]
term_to_major_rate = np.asarray(ratio.argmax(axis=0))[0]
sample_to_major_rate = np.asarray([term_to_major_rate[idx] for idx in keywords])

print(np.bincount(rows, minlength=10))
print(keywords.shape)

[684 175 296 494 200 438 384 546 479 484]
(1525,)


각 점수 별로 해당 점수대에서만 자주 등장한 단어들을 살펴봅니다. 점수를 반영하는 단어들이 선택되었음을 확인할 수 있습니다.

In [36]:
for rate in range(10):
    idxs = np.where(rows == rate_to_idx[rate+1])[0]
    terms = [idx_to_vocab[w] for w in cols[idxs]]
    terms = [w for w in terms if not ('/' in w) or ('/VA' in w) or ('/NA' in w)]
    print('\nRate #{}'.format(rate + 1))
    print(' '.join(terms))


Rate #1
영화 없/VA 것 아깝/VA 돈 이렇/VA 분 재미없/VA 알바 볼 눈 다 뻔하/VA 높/VA 음악 기억 친구 이다 드라마 존 하 오 ㅡㅡ/NA 절대 노 미 걍/NA 엄마 일본 머리 멀/VA 트랜스포머 싫/VA 에 미국 네이버 거지 낫/VA ㅡ/NA 문제 정신 어설프/VA 유머 기 라 이제 심하/VA 어이없/VA 반 다시 마라 여름 계속 못 헐리우드 지겹/VA 대통령 더럽/VA ㅉㅉ/NA 똑같/VA 생애 부끄럽/VA 유 지구 게임 과학 옛날 지나치/VA 소녀 베이 동물 맥스 ㅇㅇ/NA 소 조선 포 사이 체 두근두근 ㅅㅂ/NA 서울 인도 ㅋㅋㅋㅋㅋㅋ/NA 독립 친 리가 쩝/NA 샘 록 아이돌 민 마이클 ㅅ/NA 잘 화 휴가 시끄럽/VA 빈 청소년 월 인터넷 기타 역겹/VA 그린 ㅋㅋㅋㅋㅋㅋㅋ/NA 정보 보통 안되/VA 히말라야 쓸데없/VA 넌 못되/VA 그날 왤케/NA 다큐멘터리 ㄱ/NA 고사 두렵/VA 형편없/VA 스피드 아동 매트릭스 비싸/VA 귀찮/VA 솔/VA 일병 ㅋㅋㅋㅋㅋㅋㅋㅋ/NA 포화 데이트 이프 종교 오션스 송 진자 라스트 영광 도다 뉴스 변태 뭥미/NA 상관없/VA 소년 ㄹㅇ/NA 메인 ㄱㄱ/NA 리오 워킹 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ/NA 괴롭/VA 뷰티 ㅂ/NA 찝찝한/NA 서사 쩌네/NA ㅉ/NA 미술 잘나/VA 대부 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ/NA 성지 우습/VA 장애인 안녕 마이 집단 싱 노무현 패러디 레인 판타스틱 박근혜 구리/VA 로보트 드래곤볼

Rate #2
영화 없/VA 시간 마지막 아깝/VA 돈 분 그렇/VA 재미없/VA 반전 뻔하/VA 높/VA 인간 친구 오늘 이다 존 힘들/VA ㅡㅡ/NA 신 리 재난 걍/NA 니다 일본 멀/VA 사회 싫/VA 배경 에 미국 낫/VA ㅡ/NA 문제 어설프/VA 라 심하/VA 어이없/VA 더 지겹/VA 더럽/VA 간다 ㅉㅉ/NA 똑같/VA 맨 지나치/VA 학교 막장 ㅅㅂ/NA 착하/VA 징그럽/VA 성 정신없/VA 자동차 어지럽/VA 쩝/NA 태극기 마이클 비디오 특수 러브 로맨틱 시끄럽/VA

In [12]:
dv = doc2vec.docvecs.doctag_syn0
wv = doc2vec.wv.vectors[keywords]
print(dv.shape)
print(wv.shape)

(10, 100)
(1525, 100)


/home/lovit/anaconda3/envs/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doctag_syn0` (Attribute will be removed in 4.0.0, use docvecs.vectors_docs instead).
  """Entry point for launching an IPython kernel.


그러나 각 점수 벡터와 가까운 단어를 탐색하면 점수의 문맥이 잘 반영되지 않습니다.

In [13]:
from sklearn.metrics import pairwise_distances

def most_similar_words_from_doc(doc_idx, wv, dv, idx_to_vocab, topk=10):
    qvec = dv[doc_idx].reshape(1,-1)
    dist = pairwise_distances(wv, qvec, metric='cosine').reshape(-1)
    sim_idx = dist.argsort()[:topk]
    sims = [(idx_to_vocab[idx], 1-dist[idx]) for idx in sim_idx]
    return sims

In [14]:
most_similar_words_from_doc(0, wv, dv, idx_to_vocab)

[('약하/VA', 0.4933100938796997),
 ('광주', 0.48174619674682617),
 ('어찌/MAG', 0.4697688817977905),
 ('핵', 0.4608345031738281),
 ('후', 0.4477922320365906),
 ('베', 0.4416848421096802),
 ('세상', 0.4369851350784302),
 ('면', 0.42610329389572144),
 ('대/XPN', 0.4228662848472595),
 ('돈', 0.41721951961517334)]

In [15]:
most_similar_words_from_doc(1, wv, dv, idx_to_vocab)

[('약하/VA', 0.5206320285797119),
 ('희망', 0.5055067539215088),
 ('예상', 0.4866304397583008),
 ('해서', 0.4529011845588684),
 ('완성도', 0.4481898546218872),
 ('실화', 0.4479062557220459),
 ('글', 0.44542574882507324),
 ('사', 0.4426683187484741),
 ('핵', 0.43897581100463867),
 ('야', 0.4341089129447937)]

단어 벡터는 제대로 학습되었습니다.

In [16]:
doc2vec.wv.most_similar('영화')

/home/lovit/anaconda3/envs/jupyter/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('애니', 0.6737406849861145),
 ('애니메이션', 0.6634272336959839),
 ('연화', 0.6488072872161865),
 ('영호', 0.6288629770278931),
 ('양화', 0.6096193790435791),
 ('작품', 0.5855836272239685),
 ('여화', 0.5681470036506653),
 ('영황', 0.5434409379959106),
 ('영화ㅜㅜ/NA', 0.5325713753700256),
 ('애니매이션', 0.5255165100097656)]

In [17]:
doc2vec.wv.most_similar('재미')

/home/lovit/anaconda3/envs/jupyter/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('자미', 0.6302722692489624),
 ('임팩트', 0.6285058259963989),
 ('멋', 0.6225246787071228),
 ('잔재미', 0.6188133955001831),
 ('살수', 0.6153638362884521),
 ('재민', 0.5989111661911011),
 ('짜임새', 0.5966169834136963),
 ('품위', 0.5770062804222107),
 ('가치', 0.5667611360549927),
 ('현실감', 0.5660486221313477)]

In [18]:
doc2vec.wv.most_similar('지루')

/home/lovit/anaconda3/envs/jupyter/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('유치', 0.7604963779449463),
 ('루즈', 0.7366145253181458),
 ('지루/XR', 0.6861859560012817),
 ('지류', 0.6744557619094849),
 ('자루', 0.6427088379859924),
 ('잔인', 0.6309890747070312),
 ('방심', 0.6222023963928223),
 ('졸리/VV', 0.621128499507904),
 ('민망', 0.6142449975013733),
 ('시시', 0.6025422811508179)]

점수 벡터 역시 제대로 학습되었습니다. 0 번 document id 는 1 점 이었습니다. 이는 점수 벡터와 단어 벡터의 공간이 사실 상관이 없다는 의미입니다.

In [19]:
doc2vec.docvecs.most_similar(0)

/home/lovit/anaconda3/envs/jupyter/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('#2', 0.9795270562171936),
 ('#3', 0.9636088013648987),
 ('#4', 0.9461350440979004),
 ('#5', 0.9037704467773438),
 ('#6', 0.8336068391799927),
 ('#7', 0.7044011354446411),
 ('#10', 0.5990678071975708),
 ('#8', 0.5922185182571411),
 ('#9', 0.5608216524124146)]

PCA 를 이용하여 점수 벡터만 2 차원으로 임베딩하여 살펴보면 점수별로 연속성을 지니는 것을 알 수 있습니다.

In [20]:
from joint_visualization import annotation
from joint_visualization import draw
from joint_visualization import embedding
from bokeh.plotting import output_notebook, output_file, reset_output, show

output_notebook()
z_dv = embedding(dv, method='pca')
p = draw(z_dv, idx_to_doc, title='Rate vectors (PCA)')
annotation(p, z_dv, idx_to_doc, text_font_size='15pt')
show(p)

Loading BokehJS ...

t-SNE 는 default perplexity = 30 으로 설정되어 있는데, 데이터의 개수가 10 개 뿐이어서 30 은 지나치게 큰 값입니다. 그 결과 임베딩 공간이 제대로 학습되지 않습니다.

In [21]:
z_dv = embedding(dv, method='tsne')
p = draw(z_dv, idx_to_doc, title='Rate vectors (t-SNE)')
annotation(p, z_dv, idx_to_doc, text_font_size='15pt')
show(p)

Perplexity 만 작게 설정하여도 점수 간의 상관성이 제대로 학습되었음을 알 수 있습니다. 단, t-SNE 는 주로 barnes hut t-SNE 를 이용하는데, 이는 2 차원 공간을 원형으로 만드는 경향이 있습니다. 그렇기 때문에 10 개의 데이터를 임베딩 하기에는 좋은 방법이 아닙니다.

In [22]:
z_dv = embedding(dv, method='tsne', perplexity=3)
p = draw(z_dv, idx_to_doc, title='Rate vectors (t-SNE, perplexity=3)')
annotation(p, z_dv, idx_to_doc, text_font_size='15pt')
show(p)

혹시 단어들이 각 점수대별로 모여있는지 궁금하기도 합니다 (사실 그렇게 될 일은 없습니다). 가장 자주 나왔던 점수대별로 색을 칠해보면 경향성이 없음을 확인할 수 있습니다.

In [23]:
import numpy as np
from bokeh.palettes import Category10

z_wv = embedding(wv, method='pca')

p = None
color_map = Category10[10]
for rate in range(10):
    idxs = np.where(sample_to_major_rate == rate)[0]
    sample_to_text = [idx_to_vocab[int(idx)] for idx in idxs]
    z_wv_ = z_wv[idxs]
    p = draw(z_wv_, sample_to_text, p=p, title='Word vectors (PCA)',
             marker_size=5, marker_color=color_map[rate])
show(p)

PCA 를 이용하여 점수와 단어를 함께 임베딩하면 점수 벡터들은 (파란색) 한 공간에 몰려 있습니다. 이는 대부분의 벡터가 단어이고, 이들이 임베딩 공간의 대부분을 차지하기 때문에 Variance 를 최대한 보존하는 PCA 입장에서는 점수 벡터의 정보를 거의 무시하게 된 것입니다. 그래서 원점 근처에 위치합니다.

In [24]:
z_merge = embedding(np.vstack([wv, dv]), 'pca')
z_wv = z_merge[:wv.shape[0]]
z_dv = z_merge[wv.shape[0]:]

p = draw(z_wv, idx_to_vocab, marker_size=6, marker_alpha=0.3, marker_color='orange',
         title='Joint visualization of words and rates from Doc2Vec (PCA)')
p = draw(z_dv, idx_to_doc, p=p, marker_size=8, marker_alpha=0.3, marker_color='blue')
show(p)

t-SNE 를 이용한다고 달라지지는 않습니다.

In [25]:
z_merge = embedding(np.vstack([wv, dv]), 'tsne')
z_wv = z_merge[:wv.shape[0]]
z_dv = z_merge[wv.shape[0]:]

p = draw(z_wv, idx_to_vocab, marker_size=6, marker_alpha=0.3, marker_color='orange',
         title='Joint visualization of words and rates from Doc2Vec (t-SNE)')
p = draw(z_dv, idx_to_doc, p=p, marker_size=8, marker_alpha=0.3, marker_color='blue')
show(p)

우리는 앞서 각 단어들이 유독 어떤 점수대에서 자주 등장하는지를 ratio 로 표현했습니다. 0.5 는 점수와 단어가 서로 상관성이 없다는 의미이므로, 이 값을 기준으로 scaling 을 하여 weight 를 만듭니다. 그리고 이 weight 를 가중치로 이용하여 점수 벡터들의 가중 평균으로 단어의 벡터를 새로 계산합니다.

In [26]:
factor = 5
weight = np.exp(factor*(ratio - 0.5)) - np.exp(-0.5 * factor)
weight = normalize(weight, axis=0, norm='l1')

z_dv = embedding(dv, method='pca')

z = np.dot(weight.transpose(), z_dv)[keywords]
z.shape

(1525, 2)

그림을 그릴 때 몇 단어들은 랜드마크 역할을 하도록 미리 설정합니다. 이들은 점수와 함께 공간 전체의 모습을 이해하는데 도움을 줄 것입니다.

In [27]:
landmarks = '반드시/MAG 절대 긴급조치 드럽/VA 그럭저럭/MAG 괜찮/VA 아깝/VA 그냥저냥/MAG 조금/MAG 돈 시간 짱/MAG 그런대로/MAG 역겹/VA 무조건/MAG 알이즈웰/NA 재밋어요/NA 꼭꼭/MAG 이제야/MAG 너무너무/MAG 안쓰럽/VA 딱히/MAG'.split()

In [28]:
p = draw(z_dv, idx_to_doc, marker_size=10, title='Joint visualization of words and rates (PCA + affinity)')
annotation(p, z_dv, idx_to_doc, text_font_size='17pt')

def use_term(term):
    if (term in landmarks) or (sample_to_vocab.index(term) > 2000):
        return False
    if not ('/' in term):
        return True
    return ('/MAG' in term) or ('/VA' in term) 

color_map = Category10[10]
for rate in range(10):
    selected_terms = np.where(sample_to_major_rate == rate)[0]
    coordinates = z[selected_terms]
    idx_to_term = np.asarray(sample_to_vocab)[selected_terms]
    p = draw(coordinates, idx_to_term, p=p, marker_size=7, marker_color=color_map[rate], marker_alpha=0.15)
    idx_to_term = [w if use_term(w) else '' for w in idx_to_term]
    annotation(p, coordinates, idx_to_term, text_font_size='7pt', text_color='grey', text_alpha=0.3)

selected_terms = [sample_to_vocab.index(w) for w in landmarks]
coordinates = z[selected_terms]
annotation(p, coordinates, landmarks, text_font_size='12pt', text_color='grey', text_alpha=0.9)

단어들이 점수 공간의 벡터로 표현된 그림입니다.

In [29]:
p.height = 1000
p.width = 1500
show(p)

In [30]:
reset_output()
output_file('./joint_visualization_word_doc_movie_pca_affinity.html')
show(p)